In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

# === Last inn datasett ===
df = pd.read_csv('/Users/biraveennedunchelian/Documents/Masteroppgave/Masteroppgave/Newest addition/sinusoid curve/sinusoidal_log1idk.csv')  # Last inn hele datasettet
data = df['Y'].values.reshape(-1, 1)

# === Normaliser verdier ===
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data)

# === Lag delta (trend) ===
delta = np.diff(data_normalized, axis=0)
delta = np.vstack([[0], delta])  # Behold samme lengde på data

# === Lag sekvenser med verdi og delta ===
def create_sequences_with_delta(values, deltas, seq_length):
    xs, ys = [], []
    for i in range(len(values) - seq_length):
        val_seq = values[i:i+seq_length]
        delta_seq = deltas[i:i+seq_length]
        x = np.hstack([val_seq, delta_seq])
        y = values[i + seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 90  # Definer antall tidssteg per sekvens
X, y = create_sequences_with_delta(data_normalized, delta, seq_length)

# === PyTorch tensors ===
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)
dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# === Vanilla LSTM modell ===
class LSTMModel(nn.Module):
    def __init__(self, input_size=2, hidden_layer_size=128, output_size=1):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_layer_size, batch_first=True)
        self.linear = nn.Linear(hidden_layer_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.linear(lstm_out[:, -1, :])  # Bruk siste LSTM-utgang
        return out

# === Init modell, loss-function, optimizer ===
model = LSTMModel()
loss_function = nn.L1Loss()  # L1 Loss for mer robust prediksjon
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# === Tren modellen ===
epochs = 200
for i in range(epochs):
    for seq, labels in dataloader:
        optimizer.zero_grad()
        y_pred = model(seq)
        loss = loss_function(y_pred, labels)
        loss.backward()
        optimizer.step()
    if i % 10 == 0:
        print(f"Epoch {i} | Loss: {loss.item():.8f}")

# === Prediksjon frem i tid ===
model.eval()
last_vals = data_normalized[-seq_length:]
last_deltas = delta[-seq_length:]
future_input = np.hstack([last_vals, last_deltas]).tolist()

future_steps = 3600  # Antall fremtidige steg å predikere
future_preds = []

for i in range(future_steps):
    seq_input = torch.tensor(future_input[-seq_length:], dtype=torch.float32).unsqueeze(0)
    with torch.no_grad():
        pred = model(seq_input).item()

    prev_val = future_input[-1][0]
    new_delta = np.clip(pred - prev_val, -0.1, 0.1)

    # Støy som avtar gradvis
    decaying_noise = np.random.normal(0, 0.01 * (1 - i / future_steps))
    pred_with_noise = np.clip(pred + decaying_noise, 0.0, 1.0)

    future_preds.append(pred_with_noise)
    future_input.append([pred_with_noise, new_delta])

# === Inverse transform for å få tilbake CPU-bruk (%) ===
real_preds = scaler.inverse_transform(np.array(future_preds).reshape(-1, 1))
original_real = scaler.inverse_transform(data_normalized)

# === Plot ===
plt.figure(figsize=(15, 5))
plt.plot(original_real, label='Faktisk data')
plt.plot(range(len(original_real), len(original_real) + len(real_preds)), real_preds, label='Fremtidsprediksjon (3600 steg)')
plt.title("Vanilla LSTM Fremtidsprediksjon – Full kurve")
plt.xlabel("Tidssteg")
plt.ylabel("CPU-bruk (%)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
